In [18]:
import numpy as np
import matplotlib.pyplot as plt
from typing import Callable, Tuple

Метод золотого сечения

In [ ]:
def golden_ratio_method(
    f: Callable[[float], float], a: float, b: float, epsilon: float = 1e-8, max_iter: int = 1000
) -> Tuple[float, float]:

    K = (np.sqrt(5) - 1) / 2
    L0 = b - a
    L1 = K * L0
    x = b - L1
    y = a + L1
    
    fx, fy= f(x), f(y)
    
    for k in range(max_iter):
        L_next = K * (b - a)
        
        if fx < fy:
            b = y
            y = x
            fy = fx
            x = b - L_next
            fx = f(x)
        else:
            a = x
            x = y
            fx = fy
            y = a + L_next
            fy = f(y)
        
        if L_next < epsilon:
            break
    

    x_min, f_min = x, fx if fx < fy else y, fy
    return x_min, f_min


Метод парабол

In [ ]:
# Явная формула
def parabola_method(
    f: Callable[[float], float], a: float, c: float, b: float, epsilon: float = 1e-8, max_iter: int = 1000
) -> Tuple[float, float]:
    
    fa, fc, fb = f(a), f(c), f(b)
    
    for k in range(max_iter):
        u = c - ( (c - a)**2 * (f(c) - f(b)) - (c - b)**2 * (f(c) - f(a)) ) /  \
            ( 2 * ((c - a) * (f(c) - f(b)) - (c - b) * (f(c) - f(a))) )
        fu = f(u)

        if u < c:
            if fu < fc:
                b, fb = c, fc
                c, fc = u, fu
            elif fu > fc:
                a, fa = u, fu
        elif u > c:
            if fu < fc:
                a, fa = c, fc
                c, fc = u, fu
        else:
            b, fb = u, fu
        
        if abs(a - c) < epsilon:
            break
    
    x_min = a if fa < fb else b
    return x_min, f(x_min)

# numpy.linalge.solve
def parabola_method_numpy   (
    f: Callable[[float], float], a: float, c: float, b: float, epsilon: float = 1e-8, max_iter: int = 1000
) -> Tuple[float, float]:
    
    fa, fc, fb = f(a), f(c), f(b)
    
    for k in range(max_iter):
        A = np.array([
            [a**2, a, 1],
            [c**2, c, 1],
            [b**2, b, 1]
        ])
        B = np.array([fa, fc, fb])
        a_coeff, b_coeff, _ = np.linalg.solve(A, B)
        u = -b_coeff / (2 * a_coeff) 
        fu = f(u)

        if u < c:
            if fu < fc:
                b, fb = c, fc
                c, fc = u, fu
            elif fu > fc:
                a, fa = u, fu
        elif u > c:
            if fu < fc:
                a, fa = c, fc
                c, fc = u, fu
        else:
            b, fb = u, fu
        
        if abs(a - c) < epsilon:
            break
    
    x_min = a if fa < fb else b
    return x_min, f(x_min)

Метод Брента

In [ ]:
def brent_method(
    f: Callable[[float], float], a: float, b: float, c: float, epsilon: float = 1e-8, max_iter: int = 1000
) -> Tuple[float, float]:
    x = w = v = c
    fx = fw = fv = f(c)
    K = (np.sqrt(5) - 1) / 2
    
    for k in range(max_iter):
        u = None

        if len({fx, fw, fv}) == 3:
            A = np.array([
                [x**2, x, 1],
                [w**2, w, 1],
                [v**2, v, 1]
            ])
            B = np.array([fx, fw, fv])
            a_coeff, b_coeff, _ = np.linalg.solve(A, B)
            u = -b_coeff / (2 * a_coeff)
                


        if np.isnan(u) or u < a or u > b or abs(u - x) > abs(v - b)/2:
            # Шаг золотого сечения
            if x < (a + b) / 2:
                u = x + K * (b - x)
            else:
                u = x - K * (x - a)

        fu = f(u)

        # Обновление границ интервала
        if fu < fx:
            if u < x:
                b = x
            else:
                a = x
            v, fv = w, fw
            w, fw = x, fx
            x, fx = u, fu
        else:
            if u < x:
                a = u
            else:
                b = u
            if fu <= fw or w == x:
                v, fv = w, fw
                w, fw = u, fu
            elif fu <= fv or v == x or v == w:
                v, fv = u, fu

        if abs(b - a) < epsilon:
            break

    return x, fx


In [22]:
def fa(x: float):
    return -5*x**5 + 4*x**4 - 12*x**3 + 11*x**2 - 2*x + 1

def fc(x: float):
    return -(np.log(x-2))**2 + (np.log(10-x))**2 - x**0.2

def fb(x: float):
    return -3*x * np.sin(0.75*x) + np.exp(-2*x)

def f4(x: float):
    return np.exp(3*x) + 5*np.exp(-2*x)

def f5(x: float):
    return 0.2*x * np.log(x) + (x-2.3)**2

Явная формула:
x_min = 1.9999999999999998, f(x_min) = 4.930380657631324e-32


С использованием numpy.linalg.solve:
x_min = 1.9999999999999998, f(x_min) = 4.930380657631324e-32
